# widgets.image_cleaner

fastai offers several widgets to support the workflow of a deep learning practitioner. The purpose of the widgets are to help you organize, clean, and prepare your data for your model. Widgets are separated by data type.

In [ ]:
from fastai.vision import *
from fastai.widgets import DatasetFormatter, ImageCleaner, ImageDownloader, download_google_images
from fastai.gen_doc.nbdoc import show_doc

In [ ]:
%reload_ext autoreload
%autoreload 2

In [ ]:
path = untar_data(URLs.MNIST_SAMPLE)
data = ImageDataBunch.from_folder(path)

In [ ]:
learn = create_cnn(data, models.resnet18, metrics=error_rate)

In [ ]:
learn.fit_one_cycle(2)

epoch,train_loss,valid_loss,error_rate
1,0.177752,0.090373,0.031403
2,0.098948,0.074092,0.026006


In [ ]:
learn.save('stage-1')

We create a databunch with all the data in the training set and no validation set (DatasetFormatter uses only the training set)

In [ ]:
db = (ImageItemList.from_folder(path)
                   .no_split()
                   .label_from_folder()
                   .databunch())

In [ ]:
learn = create_cnn(db, models.resnet18, metrics=[accuracy])
learn.load('stage-1');

In [ ]:
show_doc(DatasetFormatter)

<h2 id="DatasetFormatter"><code>class</code> <code>DatasetFormatter</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L14" class="source_link">[source]</a></h2>

> <code>DatasetFormatter</code>()

Returns a dataset with the appropriate format and file indices to be displayed.  

The [`DatasetFormatter`](/widgets.image_cleaner.html#DatasetFormatter) class prepares your image dataset for widgets by returning a formatted [`DatasetTfm`](/vision.data.html#DatasetTfm) based on the [`DatasetType`](/basic_data.html#DatasetType) specified. Use `from_toplosses` to grab the most problematic images directly from your learner. Optionally, you can restrict the formatted dataset returned to `n_imgs`.

In [ ]:
show_doc(DatasetFormatter.from_similars)

<h4 id="DatasetFormatter.from_similars"><code>from_similars</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L35" class="source_link">[source]</a></h4>

> <code>from_similars</code>(<b>`learn`</b>, <b>`layer_ls`</b>:`list`=<b><i>`[0, 7, 2]`</i></b>, <b>`kwargs`</b>)

Gets the indices for the most similar images.  

In [ ]:
from fastai.gen_doc.nbdoc import *
from fastai.widgets.image_cleaner import * 

In [ ]:
show_doc(DatasetFormatter.from_toplosses)

<h4 id="DatasetFormatter.from_toplosses"><code>from_toplosses</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L16" class="source_link">[source]</a></h4>

> <code>from_toplosses</code>(<b>`learn`</b>, <b>`n_imgs`</b>=<b><i>`None`</i></b>, <b>`kwargs`</b>)

Gets indices with top losses.  

In [ ]:
show_doc(ImageCleaner)

<h2 id="ImageCleaner"><code>class</code> <code>ImageCleaner</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L96" class="source_link">[source]</a></h2>

> <code>ImageCleaner</code>(<b>`dataset`</b>, <b>`fns_idxs`</b>, <b>`path`</b>, <b>`batch_size`</b>:`int`=<b><i>`5`</i></b>, <b>`duplicates`</b>=<b><i>`False`</i></b>)

Displays images for relabeling or deletion and saves changes in `path` as 'cleaned.csv'.  

[`ImageCleaner`](/widgets.image_cleaner.html#ImageCleaner) is for cleaning up images that don't belong in your dataset. It renders images in a row and gives you the opportunity to delete the file from your file system. To use [`ImageCleaner`](/widgets.image_cleaner.html#ImageCleaner) we must first use `DatasetFormatter().from_toplosses` to get the suggested indices for misclassified images.

In [ ]:
ds, idxs = DatasetFormatter().from_toplosses(learn)

In [ ]:
ImageCleaner(ds, idxs, path)

Button(button_style='primary', description='Next Batch', layout=Layout(width='auto'), style=ButtonStyle())

[`ImageCleaner`](/widgets.image_cleaner.html#ImageCleaner) does not change anything on disk (neither labels or existence of images). Instead, it creates a 'cleaned.csv' file in your data path from which you need to load your new databunch for the files to changes to be applied. 

In [ ]:
df = pd.read_csv(path/'cleaned.csv', header='infer')

In [ ]:
# We create a databunch from our csv. We include the data in the training set and we don't use a validation set (DatasetFormatter uses only the training set)
np.random.seed(42)
db = (ImageItemList.from_df(df, path)
                   .no_split()
                   .label_from_df()
                   .databunch(bs=64))

In [ ]:
learn = create_cnn(db, models.resnet18, metrics=error_rate)
learn = learn.load('stage-1')

You can then use [`ImageCleaner`](/widgets.image_cleaner.html#ImageCleaner) again to find duplicates in the dataset. To do this, you can specify `duplicates=True` while calling ImageCleaner after getting the indices and dataset from `.from_similars`. Note that if you are using a layer's output which has dimensions <code>(n_batches, n_features, 1, 1)</code> then you don't need any pooling (this is the case with the last layer). The suggested use of `.from_similars()` with resnets is using the last layer and no pooling, like in the following cell.

In [ ]:
ds, idxs = DatasetFormatter().from_similars(learn, layer_ls=[0,7,1], pool=None)

Getting activations...


Computing similarities...


In [ ]:
ImageCleaner(ds, idxs, path, duplicates=True)

Button(button_style='primary', description='Next Batch', layout=Layout(width='auto'), style=ButtonStyle())

In [ ]:
show_doc(ImageDownloader)

<h2 id="ImageDownloader"><code>class</code> <code>ImageDownloader</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_downloader.py#L16" class="source_link">[source]</a></h2>

> <code>ImageDownloader</code>(<b>`path`</b>:`PathOrStr`=<b><i>`'data'`</i></b>)

Displays a widget that allows searching and downloading images from google images search in a Jupyter Notebook or Lab. 

[`ImageDownloader`](/widgets.image_downloader.html#ImageDownloader) widget gives you a way to quickly bootstrap your image dataset without leaving the notebook. It searches and downloads images that match the search criteria and resolution / quality requirements and stores them on your filesystem within the provided `path`.

Images for each search query (or label) are stored in a separate folder within `path`. For example, if you pupulate `tiger` with a `path` setup to `./data`, you'll get a folder `./data/tiger/` with the tiger images in it.

[`ImageDownloader`](/widgets.image_downloader.html#ImageDownloader) will automatically clean up and verify the downloaded images with [`verify_images()`](/vision.data.html#verify_images) after downloading them.

In [ ]:
path = Config.data_path()/'image_downloader_data'
os.makedirs(path, exist_ok=True)
ImageDownloader(path)

After populating images with [`ImageDownloader`](/widgets.image_downloader.html#ImageDownloader), you can get a an [`ImageDataBunch`](/vision.data.html#ImageDataBunch) by calling `ImageDataBunch.from_folder(path, size=size)`, or using the data block API.

```
src = (ImageItemList.from_folder(path)
       .random_split_by_pct()
       .label_from_folder()
       .transform(get_transforms(), size=224))
db  = src.databunch(bs=16)
learn = create_cnn(db, models.resnet34, metrics=[accuracy])
learn.fit_one_cycle(3)
```

#### Downloading more than a hundred images

To fetch more than a hundred images, [`ImageDownloader`](/widgets.image_downloader.html#ImageDownloader) uses `selenium` and `chromedriver` to scroll through the Google Images search results page and scrape image URLs. They're not required as dependencies by default. If you don't have them installed on your system, the widget will show you an error message.

To install `selenium`, just `pip install selenium` in your fastai environment.

**On a mac**, you can install `chromedriver` with `brew cask install chromedriver`.

**On Ubuntu**
Take a look at the latest Chromedriver version available, then something like:

```
wget https://chromedriver.storage.googleapis.com/2.45/chromedriver_linux64.zip
unzip chromedriver_linux64.zip
```

#### Downloading images in python scripts outside Jupyter notebooks

```
path = Config.data_path()/'image_downloader_data'
download_google_images(path, 'aussie shepherd', size='>1024*768', n_images=150)
```

In [ ]:
show_doc(download_google_images)

<h4 id="download_google_images"><code>download_google_images</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_downloader.py#L78" class="source_link">[source]</a></h4>

> <code>download_google_images</code>(<b>`path`</b>:`PathOrStr`, <b>`search_term`</b>:`str`, <b>`size`</b>:`str`=<b><i>`'>400*300'`</i></b>, <b>`n_images`</b>:`int`=<b><i>`10`</i></b>, <b>`format`</b>:`str`=<b><i>`'jpg'`</i></b>, <b>`max_workers`</b>:`int`=<b><i>`4`</i></b>, <b>`timeout`</b>:`int`=<b><i>`4`</i></b>) → `FilePathList`

Search for `n_images` images on Google, matching `search_term` and `size` requirements, download them into `path`/`search_term` and verify them, using `max_workers` threads. 

Note that downloading under 100 images doesn't require any dependencies other than fastai itself, however downloading more than a hundred images [uses `selenium` and `chromedriver`](/widgets.image_cleaner.html#Downloading-more-than-a-hundred-images).

`size` can be one of:

```
'>400*300'
'>640*480'
'>800*600'
'>1024*768'
'>2MP'
'>4MP'
'>6MP'
'>8MP'
'>10MP'
'>12MP'
'>15MP'
'>20MP'
'>40MP'
'>70MP'
```

## Methods

## Undocumented Methods - Methods moved below this line will intentionally be hidden

## New Methods - Please document or move to the undocumented section

In [ ]:
show_doc(ImageCleaner.make_dropdown_widget)

<h4 id="ImageCleaner.make_dropdown_widget"><code>make_dropdown_widget</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L126" class="source_link">[source]</a></h4>

> <code>make_dropdown_widget</code>(<b>`description`</b>=<b><i>`'Description'`</i></b>, <b>`options`</b>=<b><i>`['Label 1', 'Label 2']`</i></b>, <b>`value`</b>=<b><i>`'Label 1'`</i></b>, <b>`file_path`</b>=<b><i>`None`</i></b>, <b>`layout`</b>=<b><i>`Layout()`</i></b>, <b>`handler`</b>=<b><i>`None`</i></b>)

Return a Dropdown widget with specified `handler`.  

In [ ]:
show_doc(ImageCleaner.next_batch)

<h4 id="ImageCleaner.next_batch"><code>next_batch</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L164" class="source_link">[source]</a></h4>

> <code>next_batch</code>(<b>`_`</b>)

Handler for 'Next Batch' button click. Delete all flagged images and renders next batch.  

In [ ]:
show_doc(DatasetFormatter.sort_idxs)

<h4 id="DatasetFormatter.sort_idxs"><code>sort_idxs</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L89" class="source_link">[source]</a></h4>

> <code>sort_idxs</code>(<b>`similarities`</b>)

Sorts `similarities` and return the indexes in pairs ordered by highest similarity.  

In [ ]:
show_doc(ImageCleaner.make_vertical_box)

<h4 id="ImageCleaner.make_vertical_box"><code>make_vertical_box</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L140" class="source_link">[source]</a></h4>

> <code>make_vertical_box</code>(<b>`children`</b>, <b>`layout`</b>=<b><i>`Layout()`</i></b>, <b>`duplicates`</b>=<b><i>`False`</i></b>)

Make a vertical box with [`children`](/torch_core.html#children) and `layout`.  

In [ ]:
show_doc(ImageCleaner.relabel)

<h4 id="ImageCleaner.relabel"><code>relabel</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L157" class="source_link">[source]</a></h4>

> <code>relabel</code>(<b>`change`</b>)

Relabel images by moving from parent dir with old label `class_old` to parent dir with new label `class_new`.  

In [ ]:
show_doc(DatasetFormatter.largest_indices)

<h4 id="DatasetFormatter.largest_indices"><code>largest_indices</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L81" class="source_link">[source]</a></h4>

> <code>largest_indices</code>(<b>`arr`</b>, <b>`n`</b>)

Returns the `n` largest indices from a numpy array `arr`.  

In [ ]:
show_doc(ImageCleaner.delete_image)

<h4 id="ImageCleaner.delete_image"><code>delete_image</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L182" class="source_link">[source]</a></h4>

> <code>delete_image</code>(<b>`file_path`</b>)

In [ ]:
show_doc(ImageCleaner.empty)

<h4 id="ImageCleaner.empty"><code>empty</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L185" class="source_link">[source]</a></h4>

> <code>empty</code>()

In [ ]:
show_doc(ImageCleaner.empty_batch)

<h4 id="ImageCleaner.empty_batch"><code>empty_batch</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L180" class="source_link">[source]</a></h4>

> <code>empty_batch</code>()

In [ ]:
show_doc(DatasetFormatter.comb_similarity)

<h4 id="DatasetFormatter.comb_similarity"><code>comb_similarity</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L70" class="source_link">[source]</a></h4>

> <code>comb_similarity</code>(<b>`t1`</b>:`Tensor`, <b>`t2`</b>:`Tensor`, <b>`kwargs`</b>)

Computes the similarity function between each embedding of `t1` and `t2` matrices.  

In [ ]:
show_doc(ImageCleaner.get_widgets)

<h4 id="ImageCleaner.get_widgets"><code>get_widgets</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L188" class="source_link">[source]</a></h4>

> <code>get_widgets</code>(<b>`duplicates`</b>)

Create and format widget set.  

In [ ]:
show_doc(ImageCleaner.write_csv)

<h4 id="ImageCleaner.write_csv"><code>write_csv</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L208" class="source_link">[source]</a></h4>

> <code>write_csv</code>()

In [ ]:
show_doc(ImageCleaner.create_image_list)

<h4 id="ImageCleaner.create_image_list"><code>create_image_list</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L146" class="source_link">[source]</a></h4>

> <code>create_image_list</code>(<b>`dataset`</b>, <b>`fns_idxs`</b>)

Create a list of images, filenames and labels but first removing files that are not supposed to be displayed.  

In [ ]:
show_doc(ImageCleaner.render)

<h4 id="ImageCleaner.render"><code>render</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L219" class="source_link">[source]</a></h4>

> <code>render</code>()

Re-render Jupyter cell for batch of images.  

In [ ]:
show_doc(DatasetFormatter.get_similars_idxs)

<h4 id="DatasetFormatter.get_similars_idxs"><code>get_similars_idxs</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L41" class="source_link">[source]</a></h4>

> <code>get_similars_idxs</code>(<b>`learn`</b>, <b>`layer_ls`</b>, <b>`kwargs`</b>)

Gets the indices for the most similar images in `ds_type` dataset  

In [ ]:
show_doc(ImageCleaner.on_delete)

<h4 id="ImageCleaner.on_delete"><code>on_delete</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L175" class="source_link">[source]</a></h4>

> <code>on_delete</code>(<b>`btn`</b>)

Flag this image as delete or keep.  

In [ ]:
show_doc(ImageCleaner.make_button_widget)

<h4 id="ImageCleaner.make_button_widget"><code>make_button_widget</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L116" class="source_link">[source]</a></h4>

> <code>make_button_widget</code>(<b>`label`</b>, <b>`file_path`</b>=<b><i>`None`</i></b>, <b>`handler`</b>=<b><i>`None`</i></b>, <b>`style`</b>=<b><i>`None`</i></b>, <b>`layout`</b>=<b><i>`Layout(width='auto')`</i></b>)

Return a Button widget with specified `handler`.  

In [ ]:
show_doc(ImageCleaner.make_img_widget)

<h4 id="ImageCleaner.make_img_widget"><code>make_img_widget</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L111" class="source_link">[source]</a></h4>

> <code>make_img_widget</code>(<b>`img`</b>, <b>`layout`</b>=<b><i>`Layout()`</i></b>, <b>`format`</b>=<b><i>`'jpg'`</i></b>)

Returns an image widget for specified file name `img`.  

In [ ]:
show_doc(DatasetFormatter.get_actns)

<h4 id="DatasetFormatter.get_actns"><code>get_actns</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L52" class="source_link">[source]</a></h4>

> <code>get_actns</code>(<b>`learn`</b>, <b>`hook`</b>:[`Hook`](/callbacks.hooks.html#Hook), <b>`dl`</b>:[`DataLoader`](https://pytorch.org/docs/stable/data.html#torch.utils.data.DataLoader), <b>`pool`</b>=<b><i>`'AdaptiveConcatPool2d'`</i></b>, <b>`pool_dim`</b>:`int`=<b><i>`4`</i></b>, <b>`kwargs`</b>)

Gets activations at the layer specified by `hook`, applies `pool` of dim `pool_dim` and concatenates  

In [ ]:
show_doc(ImageCleaner.batch_contains_deleted)

<h4 id="ImageCleaner.batch_contains_deleted"><code>batch_contains_deleted</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L202" class="source_link">[source]</a></h4>

> <code>batch_contains_deleted</code>()

Check if current batch contains already deleted images.  

In [ ]:
show_doc(ImageCleaner.make_horizontal_box)

<h4 id="ImageCleaner.make_horizontal_box"><code>make_horizontal_box</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L135" class="source_link">[source]</a></h4>

> <code>make_horizontal_box</code>(<b>`children`</b>, <b>`layout`</b>=<b><i>`Layout()`</i></b>)

Make a horizontal box with [`children`](/torch_core.html#children) and `layout`.  

In [ ]:
show_doc(DatasetFormatter.get_toplosses_idxs)

<h4 id="DatasetFormatter.get_toplosses_idxs"><code>get_toplosses_idxs</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L22" class="source_link">[source]</a></h4>

> <code>get_toplosses_idxs</code>(<b>`learn`</b>, <b>`n_imgs`</b>, <b>`kwargs`</b>)

Sorts `ds_type` dataset by top losses and returns dataset and sorted indices.  

In [ ]:
show_doc(DatasetFormatter.padded_ds)

<h4 id="DatasetFormatter.padded_ds"><code>padded_ds</code><a href="https://github.com/fastai/fastai/blob/master/fastai/widgets/image_cleaner.py#L31" class="source_link">[source]</a></h4>

> <code>padded_ds</code>(<b>`ll_input`</b>, <b>`size`</b>=<b><i>`(250, 300)`</i></b>, <b>`resize_method`</b>=<b><i>`<ResizeMethod.CROP: 1>`</i></b>, <b>`padding_mode`</b>=<b><i>`'zeros'`</i></b>, <b>`kwargs`</b>)

For a LabelList `ll_input`, resize each image to `size` using `resize_method` and `padding_mode`.  